In [ ]:
!pip install datasets
from datasets import load_dataset
# https://drive.google.com/file/d/1NuPwE8wsejzFzOSxkRf8MqcFlO6q_5AQ/view?usp=sharing SST model
# https://drive.google.com/file/d/1T2OzLALdQdU-TD5nm7gs97I_5IZ271ex/view?usp=sharing NLI mode;
# dataset_m = load_dataset("multi_nli")

In [ ]:

dataset_sst = load_dataset("sst", "default")


In [3]:
print(dataset_sst)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})


In [4]:
import csv 
fields = ['text','label']
ft=open('sst_train.csv','w')
fv=open('sst_valid.csv','w')
fte=open('sst_test.csv','w')
raw_train = dataset_sst["train"].filter(lambda x: x["label"] != -1)
raw_valid = dataset_sst["validation"].filter(lambda x: x["label"] != -1)
raw_test = dataset_sst["test"].filter(lambda x: x["label"] != -1)

csvwriter=csv.writer(ft)
csvwriter.writerow(fields)
i=0
for element in raw_train:
  csvwriter.writerow([element['sentence'],element['label']])

csvwriter=csv.writer(fv)
csvwriter.writerow(fields)
i=0
for element in raw_valid:
  csvwriter.writerow([element['sentence'],element['label']])

csvwriter=csv.writer(fte)
csvwriter.writerow(fields)
i=0
for element in raw_test:
  csvwriter.writerow([element['sentence'],element['label']])

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1101 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [118]:
import nltk
import re
import torch
import torchtext
from torchtext.data import get_tokenizer
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
tokenizer = get_tokenizer("basic_english")

nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

i=0
x_train=[]
y_train=[]
#  x_train.append(row[0])
#  y_train.append(row[1])

def clean(text):
  s=s.lower()
  temp=re.sub("\s+"," ",temp)

  """
  other tokens like can't , aren't ..can also be replaced
  """
  return temp
  
def tokenise(s):
  """
  return list of tokens 
  """
  s=s.lower()
  s=re.sub("\.|`|'|,|\?","",s)
  result=re.findall("<\w+>|\w+|[\.,\"\?\:\;']",s)
  return result

def generate_vocab(filepath):
  #tokenise
  #remove stopwords
  #create dictionaries
  #returns word2Idx, Idx2word 
  stop_words = set(stopwords.words('english'))
  i=0
  tokens=[]
  with open(filepath,'r') as f:
    csvreader = csv.reader(f)
    header = next(csvreader)

    for row in csvreader:
      text=tokenise(row[0]) 
      text=[[word] for word in text if word not in stop_words]
      tokens+=text

      # text=filter(None, row[0].split('.')) 
      # # clean, tokenise, I dont think cleaning is necessary
      # #Lets just tokenise
      # for sen in text:
      #   temp=tokenise(sen)
      #   tokens+=[[word] for word in temp]
      #   print(temp)
        # tokens=[token for token in tokens if token not in stop_words]
        # print(tokens)
      # i+=1
      # if i==10000:
      #   break
  
  vocab=build_vocab_from_iterator(tokens,min_freq=1,specials=['<UNK>','<BOS>','<EOS>'],special_first=False)
  vocab.set_default_index(vocab.get_stoi()['<UNK>'])
  vocab.append_token('<pad>')
  idx2word={i:word for word,i in vocab.get_stoi().items()}
  print(vocab.get_stoi())
  print(idx2word)
  return vocab, idx2word

vocab,idx2word=generate_vocab('./sst_train.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'<pad>': 15192, '<BOS>': 15190, '<UNK>': 15189, 'élan': 15188, 'zzzzzzzzz': 15187, 'zucker': 15185, 'zoom': 15184, 'zoe': 15181, 'zinger': 15175, 'zigzag': 15174, 'zhao': 15173, 'zeus': 15172, 'zest': 15171, 'zen': 15169, 'zellweger': 15167, 'zeitgeist': 15165, 'zealously': 15164, 'zealanders': 15163, 'ze': 15162, 'zaza': 15161, 'zaidan': 15159, 'yvan': 15158, 'yung': 15157, 'yuen': 15156, 'yosuke': 15155, 'yong': 15154, 'yield': 15149, 'yesteryear': 15148, 'yelling': 15146, 'ye': 15143, 'yasujiro': 15141, 'yale': 15137, 'yakusho': 15136, 'yahoo': 15135, 'xmas': 15134, 'wyman': 15132, 'ww': 15131, 'wrinkles': 15125, 'wretchedly': 15124, 'wrestler': 15122, 'wreckage': 15120, 'wreaks': 15119, 'wrath': 15116, 'wrapping': 15115, 'wrap': 15114, 'worried': 15110, 'workmanlike': 15108, 'workman': 15107, 'workings': 15106, 'workaday': 15104, 'wordy': 15102, 'wordless': 15101, 'woodman': 15096, 'wonderous': 15091, 'wonderment': 15090, 'wladyslaw': 15086, 'wizardry': 15084, 'wizard': 15083, 'wi

In [6]:
text="The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal ."
print(tokenise(text))

['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', 's', 'new', 'conan', 'and', 'that', 'he', 's', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', 'jean', 'claud', 'van', 'damme', 'or', 'steven', 'segal']


In [7]:
from torchtext.vocab import GloVe
glove = GloVe(name='6B', dim=100)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.33MB/s]                           
100%|█████████▉| 399999/400000 [00:23<00:00, 16787.07it/s]


In [119]:
from torch.nn.utils.rnn import pad_sequence
from torch import nn, tensor
import math
class dataset(Dataset):
    def __init__(self,path,vocab,idx2word):
      self.vocab=vocab
      self.idx2word=idx2word
      sentences=[]
      labels=[]
      stop_words = set(stopwords.words('english'))
      with open(path,'r') as f:
        csvreader = csv.reader(f)
        header = next(csvreader)
        i=0
        for row in csvreader:
          labels.append(row[1])
          text=['<BOS>']
          for word in tokenise(row[0]):
            if word not in stop_words:
              text+=[word]
          text+=tokenise(row[0]) 
          text+=['<EOS>']

          modified=torch.LongTensor([vocab[word] for word in text])
          sentences.append(modified)
          # clean, tokenise, I dont think cleaning is necessary
          #Lets just tokenise
          
          # i+=1
          # if i==1000:
          #   break

      self.sentences=sentences
      self.labels=labels
      print(f"Length of sentences {len(self.sentences)}")
      print(f"Length of labels {len(self.labels)}")
      
    def __len__(self) :
      return len(self.sentences)
      
    def __getitem__(self, index : int): 
      try:
        _=[self.sentences[index],self.labels[index] ]
        # print(f"Success index {index}")
        return [self.sentences[index],self.labels[index] ]
      except:
        print(index)

def custom_collate(batch):
  # print(f"Length of Batch {len(batch)}\n")
  inputs=[inp[0] for inp in batch]
  labels=[round(float(inp[1])) for inp in batch]

  inputs=pad_sequence(inputs, batch_first=True)
  # print(f"Length of Inputs {inputs.shape} inputs[0] {len(inputs[0])} Length of labels {len(labels)}")
  
  return [inputs,labels]


def gen_embed_matrix(idx2word,embed_dim):
  vocab_size=len(idx2word)
  embed_matrix=torch.zeros(vocab_size,embed_dim)
  for index,word in idx2word.items():
        if word in glove.stoi:
            embed_matrix[index] = glove.vectors[glove.stoi[word]]
        else:
            print(f"{word}\n")
  return embed_matrix

text_data=dataset('./sst_train.csv',vocab,idx2word)#along with the labels
print(text_data[1])
train_dataloader=DataLoader(text_data, batch_size=64, shuffle=True, collate_fn=custom_collate)



Length of sentences 8482
Length of labels 8482
[tensor([15190,  3022,  1895,  6042,  3863,  2682,  7878,   785,  5985,   675,
         3416,  2174,   539,   138,    14,   995,   788,  6357,   562, 11516,
         7843,   335,  1239, 15189,  3022,  1895,  6042, 15189, 15189,  3863,
        15189, 15189,  2682,  7878, 15189, 15189,   785, 15189, 15189,  5985,
        15189,   675, 15189, 15189,  3416,  2174,   539,   138,    14,   995,
          788, 15189,  6357,   562, 15189, 11516,  7843, 15189,   335,  1239,
        15191]), '0.833329975605011']


In [146]:
val_data=dataset('./sst_valid.csv',vocab,idx2word)
test_data=dataset('./sst_test.csv',vocab,idx2word)
val_dataloader=DataLoader(val_data, batch_size=64, shuffle=True, collate_fn=custom_collate)

test_dataloader=DataLoader(val_data, batch_size=64, shuffle=True, collate_fn=custom_collate)

Length of sentences 1049
Length of labels 1049
Length of sentences 2174
Length of labels 2174


In [ ]:
embed_dim=100
class ELMo(nn.Module):
  def __init__(self, vocab_size, embedding_dim,hidden_size,embed_matrix):
    super(ELMo, self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.hidden_dim=hidden_size
    self.embeddings = nn.Embedding.from_pretrained(embed_matrix)
    self.c0 = torch.randn(2, 3, 20)
    self.f1=nn.LSTM(embedding_dim,hidden_size,1,batch_first=True)
    self.f2=nn.LSTM(hidden_size,hidden_size,1,batch_first=True)

    self.b1=nn.LSTM(embedding_dim,hidden_size,1,batch_first=True)
    self.b2=nn.LSTM(hidden_size,hidden_size,1,batch_first=True)

    self.fc = nn.Linear(2*hidden_size, vocab_size)
  def forward(self,input):
    #bz x max_len_sen 
    embeds=self.embeddings(input)
    input=tensor(input)

    batch_size,len_sen,_=embeds.shape
    # Initialsed hidden layer of 1st forward lstm 
    fh0_1 = torch.zeros(1,batch_size, self.hidden_dim)
    fc0_1 = torch.zeros(1,batch_size, self.hidden_dim)
    # Initialsed hidden layer of 2nd forward lstm 
    fh0_2 = torch.zeros(1,batch_size, self.hidden_dim)
    fc0_2 = torch.zeros(1,batch_size, self.hidden_dim)
    initialised_h2=torch.zeros(batch_size,1, self.hidden_dim)
    
    # Initialsed hidden layer of 1st backward lstm 
    bh0_1 = torch.zeros(1,batch_size, self.hidden_dim)
    bc0_1 = torch.zeros(1,batch_size, self.hidden_dim)
    # Initialsed hidden layer of 2nd backward lstm 
    bh0_2 = torch.zeros(1,batch_size, self.hidden_dim)
    bc0_2 = torch.zeros(1,batch_size, self.hidden_dim)
    initialised_b2=torch.zeros(batch_size,1, self.hidden_dim)
    
    #bz x max_len_sen x embedding_dim
    hidden_frm_f1,_=self.f1(embeds,(fh0_1,fc0_1))
    hidden_frm_f2,_=self.f2(hidden_frm_f1,(fh0_2,fc0_2))
    hidden_frm_f2=hidden_frm_f2[:,:-1,:]
    assert(hidden_frm_f2.shape==(batch_size,len_sen-1,self.hidden_dim))
    # forward_hidden_states=[[fh0_2[i]]+hidden_frm_f2[i] for i in range(batch_size)]
    forward_hidden_states=torch.cat((initialised_h2,hidden_frm_f2),1)
    assert(len(forward_hidden_states[0])==len_sen)
    assert(len(forward_hidden_states)==batch_size)
    assert(len(forward_hidden_states[0][0])==self.embedding_dim)



    # [w1,w2,w3,w4]-->reverse and send [w4,w3,w2,w1]
    reversed_embeds=torch.flip(embeds,[1])

    hidden_frm_b1,_=self.b1(reversed_embeds,(bh0_1,bc0_1))
    hidden_frm_b2,_=self.b2(hidden_frm_b1,(bh0_2,bc0_2))
    hidden_frm_b2=hidden_frm_b2[:,:-1,:]#removed last output since it contains all information
    # print(hidden_frm_b2.shape) #bz,len_sen-1,embedding_dim
    
    # backward_hidden_states=[[bh0_2]+hidden_frm_b2[i] for i in range(batch_size)]
    backward_hidden_states=torch.cat((initialised_b2,hidden_frm_b2),1)
    backward_hidden_states=torch.flip(backward_hidden_states,[1])
    # print(backward_hidden_states.shape) #bz,len_sen,embedding_dim

    # hidden_frm_b2=torch.flip(hidden_frm_b2,[1])
    # hidden_frm_b2=hidden_frm_b2[:,1:,:]

    assert(hidden_frm_b2.shape==(batch_size,len_sen-1,self.hidden_dim))
    assert(len(backward_hidden_states[0])==len_sen)
    assert(len(backward_hidden_states)==batch_size)
    assert(len(backward_hidden_states[0][0])==self.embedding_dim)

    final_embeds=torch.cat((forward_hidden_states,backward_hidden_states),2)
    assert(final_embeds.shape==(batch_size,len_sen,2*self.hidden_dim))

    prob_dists=self.fc(final_embeds)
    # print(prob_dists.shape)
    assert(prob_dists.shape==(input.shape[0],input.shape[1],self.vocab_size))
    return prob_dists
embed_matrix=gen_embed_matrix(idx2word,embed_dim)
model=ELMo(len(vocab),embed_dim,100,embed_matrix)

In [123]:
import torch.optim as optim
vocab_size = len(vocab)
print(vocab_size)

epochs = 6
learning_rate = 0.01
# batch size is also a hyperparameter
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

15193


In [135]:
a=torch.nn.Softmax(dim=1)
for epoch in range(epochs):
  train_loss = 0
  model.train()
  all_pred=[]
  all_actual=[]
  for i,batchdata in enumerate(train_dataloader):
    batchdata=batchdata[0]
    x_train, y_train = batchdata,tensor(batchdata)
    # print(y_train.shape)
    
    optimizer.zero_grad()# clears the gradients from prev iteration 
    # print('train',x_train.shape)
    output = model(x_train) # forward pass
    output=output.view(-1,output.shape[2])
    # print(output.shape)
    output = a(output)
    target=y_train.view(-1)
    # print('test',output.shape,y_train.shape)
    loss = loss_fn(output, target) # calculate loss
    # print(loss)

    #backprop
    loss.backward() # compute gradients
    optimizer.step() # update parameters
    
    train_loss += loss.item()
  val_loss = 0
  model.eval()
  for i, batchdata in enumerate(val_dataloader):
    batchdata=batchdata[0]
    x_val, y_val = batchdata,tensor(batchdata)
    output = model(x_val)
    output=output.view(-1,output.shape[2])
    output=a(output)
    
    loss = loss_fn(output, y_val.reshape(-1))
    val_loss += loss.item()

  print(f"Epoch {epoch} TrainingLoss {train_loss/len(train_dataloader)} validationLoss {val_loss/len(val_dataloader)}\n")



<ipython-input-135-dba72e1a98a0>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train, y_train = batchdata,tensor(batchdata)
<ipython-input-122-d2524b2f286b>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input=tensor(input)
<ipython-input-135-dba72e1a98a0>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val, y_val = batchdata,tensor(batchdata)


Epoch 0 TrainingLoss 9.074008697853948 validationLoss 9.096981160780963

Epoch 1 TrainingLoss 9.074065810755679 validationLoss 9.1053466796875

Epoch 2 TrainingLoss 9.0729288660494 validationLoss 9.095699815189137

Epoch 3 TrainingLoss 9.072808760449401 validationLoss 9.098126355339499

Epoch 4 TrainingLoss 9.074557705929404 validationLoss 9.105809604420381

Epoch 5 TrainingLoss 9.071813769806596 validationLoss 9.099087378557991



In [137]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'embedding_dim': model.embedding_dim,
            'hidden_dim': model.hidden_dim,
            'vocab_size': vocab_size
            }, 'pre_trained_sst.pth')
pre_trained=torch.load('./pre_trained_sst.pth')
print(pre_trained.keys())

dict_keys(['model_state_dict', 'optimizer_state_dict', 'embedding_dim', 'hidden_dim', 'vocab_size'])


In [138]:
print(pre_trained['model_state_dict']['f1.weight_ih_l0'])

tensor([[ 0.0411,  0.1258,  0.0641,  ..., -0.0302,  0.1620, -0.0012],
        [ 0.0654,  0.0047, -0.0155,  ..., -0.0604,  0.0276,  0.0783],
        [ 0.0530,  0.0236,  0.1050,  ..., -0.0484,  0.1085,  0.1478],
        ...,
        [ 0.1386, -0.0135,  0.1141,  ..., -0.0858,  0.0342,  0.1146],
        [ 0.0571, -0.1122,  0.0016,  ..., -0.0322,  0.0865, -0.0069],
        [ 0.0949,  0.0316,  0.0193,  ..., -0.0982,  0.0930,  0.1168]])


In [75]:
embeddingwts=pre_trained['model_state_dict']['embeddings.weight']
# _(self, vocab_size, embedding_dim,hidden_size,embed_matrix)

In [139]:
class classification(nn.Module):
  def __init__(self, num_classes,pre_trained_model_path):
    super(classification, self).__init__()

    checkpoint = torch.load(pre_trained_model_path)
    self.embedding_dim=checkpoint['embedding_dim']
    self.hidden_dim=checkpoint['hidden_dim']
    self.vocab_size=checkpoint['vocab_size']

    # self.embedding_wts = pre_trained['model_state_dict']['embeddings.weight']
    self.embedding=nn.Embedding.from_pretrained(embed_matrix,freeze=True)
    self.f1=nn.LSTM(self.embedding_dim,self.hidden_dim,1,batch_first=True)
    self.f2=nn.LSTM(self.hidden_dim,self.hidden_dim,1,batch_first=True)

    self.b1=nn.LSTM(self.embedding_dim,self.hidden_dim,1,batch_first=True)
    self.b2=nn.LSTM(self.hidden_dim,self.hidden_dim,1,batch_first=True)

    self.fc = nn.Linear(self.hidden_dim, num_classes)
    self.lstm=nn.LSTM(2*self.hidden_dim,self.hidden_dim,1,batch_first=True)#output would be Batchsize x max_len_sen x hidden_size

    self.pretrained_model = ELMo(self.vocab_size,self.embedding_dim,self.hidden_dim,embed_matrix)
    self.pretrained_model.load_state_dict(checkpoint['model_state_dict'])

    self.f1.weight_ih_l0=self.pretrained_model.f1.weight_ih_l0
    self.f1.weight_hh_l0=self.pretrained_model.f1.weight_hh_l0
    self.f1.bias_ih_l0=self.pretrained_model.f1.bias_ih_l0
    self.f1.bias_ih_l0=self.pretrained_model.f1.bias_ih_l0
    self.f2.weight_ih_l0=self.pretrained_model.f2.weight_ih_l0
    self.f2.weight_hh_l0=self.pretrained_model.f2.weight_hh_l0
    self.f2.bias_ih_l0=self.pretrained_model.f2.bias_ih_l0
    self.f2.bias_ih_l0=self.pretrained_model.f2.bias_ih_l0

    self.b1.weight_ih_l0=self.pretrained_model.b1.weight_ih_l0
    self.b1.weight_hh_l0=self.pretrained_model.b1.weight_hh_l0
    self.b1.bias_ih_l0=self.pretrained_model.b1.bias_ih_l0
    self.b1.bias_ih_l0=self.pretrained_model.b1.bias_ih_l0
    self.b2.weight_ih_l0=self.pretrained_model.b2.weight_ih_l0
    self.b2.weight_hh_l0=self.pretrained_model.b2.weight_hh_l0
    self.b2.bias_ih_l0=self.pretrained_model.b2.bias_ih_l0
    self.b2.bias_ih_l0=self.pretrained_model.b2.bias_ih_l0


    self.weights = nn.Parameter(torch.tensor([0.33, 0.33, 0.33]), requires_grad=True)
  def forward(self,inputs):
    # inputs-bz, len_sen
    embeds=self.embedding(inputs)

    batch_size,len_sen,_=embeds.shape
    # Initialsed hidden layer of 1st forward lstm 
    fh0_1 = torch.zeros(1,batch_size, self.hidden_dim)
    fc0_1 = torch.zeros(1,batch_size, self.hidden_dim)
    # Initialsed hidden layer of 2nd forward lstm 
    fh0_2 = torch.zeros(1,batch_size, self.hidden_dim)
    fc0_2 = torch.zeros(1,batch_size, self.hidden_dim)
    initialised_h2=torch.zeros(batch_size,1, self.hidden_dim)
    
    # Initialsed hidden layer of 1st backward lstm 
    bh0_1 = torch.zeros(1,batch_size, self.hidden_dim)
    bc0_1 = torch.zeros(1,batch_size, self.hidden_dim)
    # Initialsed hidden layer of 2nd backward lstm 
    bh0_2 = torch.zeros(1,batch_size, self.hidden_dim)
    bc0_2 = torch.zeros(1,batch_size, self.hidden_dim)
    initialised_b2=torch.zeros(batch_size,1, self.hidden_dim)
    
    #bz x max_len_sen x embedding_dim
    hidden_frm_f1,_=self.f1(embeds,(fh0_1,fc0_1))
    hidden_frm_f2,_=self.f2(hidden_frm_f1,(fh0_2,fc0_2))
    hidden_frm_f2=hidden_frm_f2[:,:-1,:]
    hidden_frm_f1=hidden_frm_f1[:,:-1,:]
    assert(hidden_frm_f2.shape==(batch_size,len_sen-1,self.hidden_dim))
    # forward_hidden_states=[[fh0_2[i]]+hidden_frm_f2[i] for i in range(batch_size)]
    forward_hidden_states_2=torch.cat((initialised_h2,hidden_frm_f2),1)
    forward_hidden_states_1=torch.cat((initialised_h2,hidden_frm_f1),1)
    assert(len(forward_hidden_states_1[0])==len_sen)
    assert(len(forward_hidden_states_1)==batch_size)
    assert(len(forward_hidden_states_1[0][0])==self.embedding_dim)



    # [w1,w2,w3,w4]-->reverse and send [w4,w3,w2,w1]
    reversed_embeds=torch.flip(embeds,[1])

    hidden_frm_b1,_=self.b1(reversed_embeds,(bh0_1,bc0_1))
    hidden_frm_b2,_=self.b2(hidden_frm_b1,(bh0_2,bc0_2))
    hidden_frm_b2=hidden_frm_b2[:,:-1,:]#removed last output since it contains all information
    hidden_frm_b1=hidden_frm_b1[:,:-1,:]
    # print(hidden_frm_b2.shape) #bz,len_sen-1,embedding_dim
    
    # backward_hidden_states=[[bh0_2]+hidden_frm_b2[i] for i in range(batch_size)]
    backward_hidden_states_2=torch.cat((initialised_b2,hidden_frm_b2),1)
    backward_hidden_states_2=torch.flip(backward_hidden_states_2,[1])

    backward_hidden_states_1=torch.cat((initialised_b2,hidden_frm_b1),1)
    backward_hidden_states_1=torch.flip(backward_hidden_states_1,[1])
    # print(backward_hidden_states_2.shape) #bz,len_sen,embedding_dim

    # hidden_frm_b2=torch.flip(hidden_frm_b2,[1])
    # hidden_frm_b2=hidden_frm_b2[:,1:,:]

    assert(hidden_frm_b2.shape==(batch_size,len_sen-1,self.hidden_dim))
    assert(len(backward_hidden_states_2[0])==len_sen)
    assert(len(backward_hidden_states_2)==batch_size)
    assert(len(backward_hidden_states_2[0][0])==self.embedding_dim)

    layer2=torch.cat((forward_hidden_states_2,backward_hidden_states_2),2)
    layer1=torch.cat((forward_hidden_states_1,backward_hidden_states_1),2)
    layer0=torch.cat((embeds,embeds),2)
    assert(layer2.shape==(batch_size,len_sen,2*self.hidden_dim))
    assert(layer1.shape==(batch_size,len_sen,2*self.embedding_dim))
    final=(self.weights[0]*layer0+self.weights[1]*layer1+self.weights[2]*layer2)/(self.weights[0]+self.weights[1]+self.weights[2])
    out,_=self.lstm(final)
    prob_dists=self.fc(out[:,-1,:])
    return prob_dists

clmodel=classification(2,'./pre_trained_sst.pth')

In [145]:
from sklearn.metrics import classification_report
a=torch.nn.Softmax(dim=1)
for epoch in range(epochs):
  train_loss = 0
  clmodel.train()
  
  for i,batchdata in enumerate(train_dataloader):
   
    x_train, y_train = batchdata[0],tensor(batchdata[1])
    # print(y_train.shape)
    
    optimizer.zero_grad()# clears the gradients from prev iteration 
    # print('train',x_train.shape)
    output = clmodel(x_train) # forward pass
    
    output=output.view(-1,output.shape[1])
    # print(output.shape)
    output = a(output)
    target=y_train.view(-1)
    # print('test',output.shape,y_train.shape)
    loss = loss_fn(output, target) # calculate loss
    # print(loss)

    #backprop
    loss.backward() # compute gradients
    optimizer.step() # update parameters
    
    train_loss += loss.item()
  val_loss = 0
  model.eval()
  all_actual=[]
  all_pred=[]
  for i, batchdata in enumerate(val_dataloader):
    # print(len(batchdata))
    # print(len(batchdata[0]))
    x_val, y_val = batchdata[0],tensor(batchdata[1])
    output = clmodel(x_val)
    output=output.view(-1,output.shape[1])
    output=a(output)
    print(output.shape)
    target=y_val.view(-1)

    loss = loss_fn(output, target)
    output= torch.argmax(output, dim=1)
    all_pred+=output
    all_actual+=y_val.reshape(-1)
    
    val_loss += loss.item()
  print(classification_report(all_actual, all_pred))
  print(f"Epoch {epoch} TrainingLoss {train_loss/len(train_dataloader)} validationLoss {val_loss/len(val_dataloader)}\n")

torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([25, 2])


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.49      1.00      0.65       509
           1       0.00      0.00      0.00       540

    accuracy                           0.49      1049
   macro avg       0.24      0.50      0.33      1049
weighted avg       0.24      0.49      0.32      1049

Epoch 0 TrainingLoss 0.6934665965854674 validationLoss 0.6936784807373496

torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([25, 2])


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.49      1.00      0.65       509
           1       0.00      0.00      0.00       540

    accuracy                           0.49      1049
   macro avg       0.24      0.50      0.33      1049
weighted avg       0.24      0.49      0.32      1049

Epoch 1 TrainingLoss 0.6934859170053238 validationLoss 0.6935032080201542

torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([25, 2])
              precision    recall  f1-score   support

           0       0.49      1.00      0.65       509
           1       0.00      0.00      0.00       540

    accuracy                           0.49      1049
   macro avg       0.24      0.50      0

...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.49      1.00      0.65       509
           1       0.00      0.00      0.00       540

    accuracy                           0.49      1049
   macro avg       0.24      0.50      0.33      1049
weighted avg       0.24      0.49      0.32      1049

Epoch 3 TrainingLoss 0.6934907297442731 validationLoss 0.6937943135990816

torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([25, 2])
              precision    recall  f1-score   support

           0       0.49      1.00      0.65       509
           1       0.00      0.00      0.00       540

    accuracy                           0.49      1049
   macro avg       0.24      0.50      0

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([25, 2])
              precision    recall  f1-score   support

           0       0.49      1.00      0.65       509
           1       0.00      0.00      0.00       540

    accuracy                           0.49      1049
   macro avg       0.24      0.50      0.33      1049
weighted avg       0.24      0.49      0.32      1049

Epoch 5 TrainingLoss 0.6934847163974791 validationLoss 0.6937795106102439



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [149]:
for i, batchdata in enumerate(test_dataloader):
    # print(len(batchdata))
    # print(len(batchdata[0]))
    x_val, y_val = batchdata[0],tensor(batchdata[1])
    output = clmodel(x_val)
    output=output.view(-1,output.shape[1])
    output=a(output)
    print(output.shape)
    target=y_val.view(-1)

    loss = loss_fn(output, target)
    output= torch.argmax(output, dim=1)
    all_pred+=output
    all_actual+=y_val.reshape(-1)
    
    val_loss += loss.item()
print(classification_report(all_actual, all_pred))

torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([25, 2])
              precision    recall  f1-score   support

           0       0.49      1.00      0.65      1018
           1       0.00      0.00      0.00      1080

    accuracy                           0.49      2098
   macro avg       0.24      0.50      0.33      2098
weighted avg       0.24      0.49      0.32      2098



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
